In [10]:
library("dplyr")

## Inputs: 

1. Set of nodes *Z*
2. Ordering of the nodes
3. Upper bound on the number of parents (for each node) *u*
4. Database *D* of m cases

In [301]:
N_tensor <- function(D, i, V.i, pi.i){
    if (length(pi.i) <= 1){
        result <- data.frame(j_counts=nrow(D))
        for (k in V.i){
            k_count <- data.frame(nrow(D[D[,i]==k,]))
            names(k_count)<-k 
            result<-merge(result, k_count, all.x=TRUE)
        }
    }
    else {
        result<- D[,pi.i] %>% group_by_all() %>% summarise(j_count= n())
        for (k in V.i){
            k_count <- D[D[, i]==k,pi.i] %>% group_by_all() %>% summarise(name=n())
            colnames(k_count)[length(colnames(k_count))]<- k
            result<-merge(result, k_count, all.x=TRUE)
        }
        result[is.na(result)] <- 0
    }
    return(result)
}

g <- function(r.i, N.tensor){
    N.ij <- N.tensor$j_counts
    q.i <- length(N.ij)
    result <- 1
    for (j in 1:q.i) {
        a <- 1
        for (k in 0:(r.i-1)){a <- a*factorial(N.tensor[j,as.character(k)])}
        result <- result*a*factorial(r.i-1)/factorial(N.ij[j]+r.i-1)
    }
     return(result)
}


k2 <- function(Z, u, D){
    V <- unname(sapply(D, unique)) #it's a matrix
    r <- unname(sapply(D, n_distinct)) #it's a matrix
    parents <- list()
    for (i in 1:length(Z)){
        pi.i <- NULL 
        Nijk <- N_tensor(D, Z[i], V[,i], pi.i)
        P_old <- g(r[i], Nijk)
        proceed <- TRUE
        while (proceed & (length(pi.i)<u & i>1)){
            pred.i <- setdiff(Z[1:i-1], pi.i)
            print(paste('pi', pi.i))
            print(paste('pred', pred.i))
            P <- NULL
            for (node in pred.i){
                new_set <- union(pi.i, node)
                print(new_set)
                Nijk <- N_tensor(D, Z[i], V[,i], new_set)
                P<- c(P, g(r[i], Nijk))
            }
            print(P)
            if (max(P) > P_old){
                print("ok")
                max.indx <- which.max(P)
                P_old <- P[max.indx]
                pi.i <- union(pi.i, pred.i[max.indx])
            }
            else {proceed = FALSE}
        }
        append(parents, list(pi.i))
    } 
    return(parents)
}


In [302]:
x1 <- c(1,1,0,1,0,0,1,0,1,0)
x2 <- c(0,1,0,1,0,1,1,0,1,0)
x3<- c(0,1,1,1,0,1,1,0,1,0)

D <- data.frame(x1, x2, x3)
print(D)


   x1 x2 x3
1   1  0  0
2   1  1  1
3   0  0  1
4   1  1  1
5   0  0  0
6   0  1  1
7   1  1  1
8   0  0  0
9   1  1  1
10  0  0  0


In [303]:
k2(c(1,2,3), u=3, D)

[1] "pi "
[1] "pred 1"
[1] 1
[1] 0.0003607504
[1] "pi "
[1] "pred 1" "pred 2"
[1] 1
[1] 2
[1] 0.0004329004 0.0004329004


list()

In [293]:
1/900

[1] 0.001111111

In [98]:
pi.i <- c(1,2)
D[,pi.i] %>% group_by_all() %>% summarise(j_count= n())


x1,x2,j_count
0,0,4
0,1,1
1,0,1
1,1,4


In [93]:
length(NULL)

[1] 0

In [193]:
0:1

[1] 0 1

In [234]:
x <- c(2,3,4,2,1)
unique(x)

[1] 2 3 4 1